<a href="https://colab.research.google.com/github/fabiomatricardi/llamacppChatDocs/blob/main/llamacppEmbeddings%2BModel_chatWithDocs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://i.ytimg.com/vi/3H8X8q_XIJU/maxresdefault.jpg' height=300>

In [1]:
%pip install --upgrade --quiet rich tiktoken PyMuPDF langchain langchain-community llama-cpp-python faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 MB 14.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━

In [2]:
!wget https://huggingface.co/leliuga/all-MiniLM-L6-v2-GGUF/resolve/main/all-MiniLM-L6-v2.F16.gguf
!wget https://huggingface.co/Qwen/Qwen2-0.5B-Instruct-GGUF/resolve/main/qwen2-0_5b-instruct-q8_0.gguf
!wget https://github.com/fabiomatricardi/How-I-Built-a-Chatbot-that-Crushed-ChatGPT/raw/main/What%20is%20Entrepreneurship%205%20Factors%20Affecting%20Ent.txt
!wget https://huggingface.co/gaianet/Nomic-embed-text-v1.5-Embedding-GGUF/resolve/main/nomic-embed-text-v1.5.f16.gguf

--2024-06-11 19:17:34--  https://huggingface.co/leliuga/all-MiniLM-L6-v2-GGUF/resolve/main/all-MiniLM-L6-v2.F16.gguf
Resolving huggingface.co (huggingface.co)... 18.172.134.124, 18.172.134.88, 18.172.134.4, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.124|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/68/86/688665eb5b2019706f226f4e2a0cb26c8f210c781549d129dc7f26d630ac2863/797b70c4edf85907fe0a49eb85811256f65fa0f7bf52166b147fd16be2be4662?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27all-MiniLM-L6-v2.F16.gguf%3B+filename%3D%22all-MiniLM-L6-v2.F16.gguf%22%3B&Expires=1718392654&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxODM5MjY1NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzY4Lzg2LzY4ODY2NWViNWIyMDE5NzA2ZjIyNmY0ZTJhMGNiMjZjOGYyMTBjNzgxNTQ5ZDEyOWRjN2YyNmQ2MzBhYzI4NjMvNzk3YjcwYzRlZGY4NTkwN2ZlMGE0OWViODU4MTEyNTZm

directly from https://python.langchain.com/v0.2/docs/how_to/vectorstore_retriever/

In [3]:
from rich.markdown import Markdown
import warnings
warnings.filterwarnings(action='ignore')
import datetime
from rich.console import Console
console = Console(width=90)
import tiktoken
from time import sleep
encoding = tiktoken.get_encoding("r50k_base") #context_count = len(encoding.encode(yourtext))

from langchain.text_splitter import TokenTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import LlamaCppEmbeddings
from llama_cpp import Llama
from langchain_text_splitters import CharacterTextSplitter

In [4]:
embpath = "/content/all-MiniLM-L6-v2.F16.gguf"
embeddings = LlamaCppEmbeddings(model_path=embpath)

llama_model_loader: loaded meta data with 23 key-value pairs and 101 tensors from /content/all-MiniLM-L6-v2.F16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = bert
llama_model_loader: - kv   1:                               general.name str              = all-MiniLM-L6-v2
llama_model_loader: - kv   2:                           bert.block_count u32              = 6
llama_model_loader: - kv   3:                        bert.context_length u32              = 512
llama_model_loader: - kv   4:                      bert.embedding_length u32              = 384
llama_model_loader: - kv   5:                   bert.feed_forward_length u32              = 1536
llama_model_loader: - kv   6:                  bert.attention.head_count u32              = 12
llama_model_loader: - kv   7:          bert.attention.layer_norm_epsilon f

In [5]:
from llama_cpp import Llama
qwen05b = Llama(
            model_path='/content/qwen2-0_5b-instruct-q8_0.gguf',
            n_gpu_layers=0,
            temperature=0.1,
            top_p = 0.5,
            n_ctx=8192,
            max_tokens=600,
            repeat_penalty=1.7,
            stop=["<|im_end|>","Instruction:","### Instruction:","###<user>","</user>"],
            verbose=False,
            )

In [7]:
# load a TXT document
loader = TextLoader("/content/What is Entrepreneurship 5 Factors Affecting Ent.txt")
#Create a document and split into chuncks
documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size=250, chunk_overlap=50)
texts = text_splitter.split_documents(documents)
#create the vector store
vectorstore = FAISS.from_documents(texts, embeddings)

In [ ]:
console.print(documents) #this will print the entire document object, without chunks

In [13]:
#default is similarity search
retriever = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": 3})

In [14]:
docs = retriever.invoke("what to do to get started with entrepreneurship?")

In [15]:
console.print(docs)

[
    Document(
        page_content=' success.\nBy carefully considering these factors, entrepreneurs can
increase their chances of navigating the challenges and maximizing the opportunities 
within the dynamic world of entrepreneurship.\nVI. Step-by-step guide to get started with 
entrepreneurship\nStep-by-step guide to get started with entrepreneurship\nThe world of 
entrepreneurship beckons with the promise of innovation, independence, and the chance to 
make a real difference. If you’re brimming with an idea and ready to embark on this 
exciting path, here’s a roadmap to guide you through the initial stages:\n1. Validate Your
Idea\nMarket Research: Don’t be afraid to get out there and talk to people! Conduct 
thorough market research to understand if there’s a',
        metadata={
            'source': '/content/What is Entrepreneurship 5 Factors Affecting Ent.txt'
        }
    ),
    Document(
        page_content=' Practices and Examples\n3. Secure Funding\nBootstrapping: Many 
entrepreneurs start by bootstrapping their businesses using personal savings, loans from 
family and friends, or crowdfunding platforms.\nSeeking Investors: For ventures requiring 
significant capital, exploring options like angel investors or venture capital firms might
be necessary. Be prepared to present a compelling pitch that highlights the potential of 
your business.\n4. Build Your Team\nAssemble Your A-Team: Surround yourself with talented 
and passionate individuals who complement your skillset. Delegate tasks effectively and 
foster a collaborative environment where everyone feels valued and empowered to 
contribute.\n5. Choose a Business Structure\nSole Proprietorship, Partnership, LLC, or 
Corporation? The legal structure you choose for your',
        metadata={
            'source': '/content/What is Entrepreneurship 5 Factors Affecting Ent.txt'
        }
    ),
    Document(
        page_content=' us. Here’s why entrepreneurship is so important:\n1. Driving 
Economic Growth: Entrepreneurs are the lifeblood of economic dynamism. By starting new 
businesses, they create jobs, stimulate competition, and introduce innovative products and
services. This fosters a healthy and growing economy for everyone.\n2. Fostering 
Innovation: Entrepreneurs are constantly pushing boundaries and seeking solutions to 
problems. Their drive to create something new leads to breakthroughs in technology, 
business models, and social approaches. This continuous innovation keeps industries 
evolving and benefits society as a whole.\n3. Job Creation: New businesses create new 
jobs. In fact, startups and small businesses are responsible for a significant portion of 
job growth in many economies. Entrepreneurship provides opportunities',
        metadata={
            'source': '/content/What is Entrepreneurship 5 Factors Affecting Ent.txt'
        }
    )
]

#### RESULTS With scores

https://python.langchain.com/v0.2/docs/how_to/add_scores_retriever/

https://python.langchain.com/v0.1/docs/integrations/vectorstores/faiss/

In [ ]:
docs_and_scores = vectorstore.similarity_search_with_score("what is Apple news?",k=3) #or .similarity_search_with_relevance_scores()


llama_print_timings:        load time =     169.52 ms
llama_print_timings:      sample time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings: prompt eval time =      15.07 ms /     7 tokens (    2.15 ms per token,   464.56 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =      16.47 ms /     8 tokens


In [ ]:
console.print(docs_and_scores)

[
    (
        Document(
            page_content='Apple\'s iOS App Store announces sweeping changes in the 
EU\nhttps://www.axios.com/2024/01/25/apple-app-store-eu-changes\nAuthor: Ashley Gold, 
author of\xa0Axios Pro\n\n\nApple will allow alternative app stores and other new options 
for app developers in order to comply with Europe\'s Digital Markets Act, the company said
Thursday.\n\nDriving the news: Europe\'s major tech competition law, set to go into effect
March 7, requires Apple to loosen its strict rules requiring developers to rely on the App
Store for distribution and payment processing.\n\nApple has long resisted any changes to 
its App Store rules, citing security and privacy. The company also argues its services for
developers are best-in-class and ensure smooth user experiences free of spam or harmful 
content.\n\nWhy it matters: Apple is being forced to make sweeping changes to one of its 
flagship "walled garden" systems in what some smaller companies are seeing as a major 
win.',
            metadata={'source': '/content/Article-edited.txt'}
        ),
        39.76284
    ),
    (
        Document(
            page_content='Go deeper: The iOS changes will include notarization for iOS 
apps, authorization for marketplace developers and disclosures on alternative payments. 
Apple says in the release that if apps opt out of Apple\'s system, the company "has less 
ability to address other risks" like "scam, fraud, abuse" or "objectionable or harmful" 
content.\n\nUsers in the EU will also see a browser choice screen in iOS 17.4 to pick a 
default browser.\n\nThe other side: Epic Games CEO Tim Sweeney said on X: "Apple\'s plan 
to thwart Europe\'s new Digital Markets Act law is a devious new instance of Malicious 
Compliance... Epic has always supported the notion of Apple notarization and malware 
scanning for apps, but we strongly reject Apple\'s twisting this process to undermine 
competition and continue imposing Apple taxes on transactions they\'re not involved in."',
            metadata={'source': '/content/Article-edited.txt'}
        ),
        40.713356
    ),
    (
        Document(
            page_content="What to watch: It's up to the European Commission to determine 
whether Apple's plan fully complies with the DMA. iPhone app developers face a whole new 
world of choices, and time will tell if the changes boost competition and improve 
experiences for users.",
            metadata={'source': '/content/Article-edited.txt'}
        ),
        41.08703
    )
]

### Query the documents and use the LLM to reply

In [8]:
#Function for QnA over a Context - the context is pure string text
def QwenQnA(question,vectorstore,hits,maxtokens,model):
  """
  basic generation with Qwen2-0.5b-Instruct/ any llama.cpp loaded model
  question -> string
  vectorstore-> a FAISS objects
  hits -> int, the number of results from the similarity search
  maxtokens -> int, number of max tokens to generate
  model -> llama-cpp-python instance // here is Qwen2-0.5b-Instruct AI
  RETURNS output -> str, docs -> list of langchain documents from similarity search
  """
  #docs_and_scores = vectorstore.similarity_search_with_score(question,k=3) #or .similarity_search_with_relevance_scores()
  retriever = vectorstore.as_retriever(search_type="mmr",search_kwargs={"k": hits})
  docs = retriever.invoke(question)
  # pack all the text from the retriever hits and remove carriage return
  context = ''
  for i in docs:
    context += i.page_content
  contesto = context.replace('\n\n','')
  query = question
  import datetime
  start = datetime.datetime.now()
  #create the prompt template
  template = f"""Answer the question based only on the following context:

[context]
{contesto}
[end of context]

Question: {query}

"""
  #structure the prompt into a chat template
  messages = [
              {"role": "system", "content": "You are a Language Model trained to answer questions based solely on the provided text.",},
              {"role": "user", "content": template},
              ]
  with console.status("Qwen-0.5b-Chat AI is working ✅✅✅ ...",spinner="dots12"):
    output = model.create_chat_completion(
                    messages=messages,
                    max_tokens=maxtokens,
                    stop=["</s>","[/INST]","/INST",'<|eot_id|>','<|end|>'],
                    temperature = 0.1,
                    repeat_penalty = 1.4)
  delta = datetime.datetime.now() - start
  console.print(f"[bright_green bold on black]Question: {query}")
  console.print(output["choices"][0]["message"]["content"])
  console.print(f"Completed in: [bold bright_red]{delta}")
  return output["choices"][0]["message"]["content"], docs

In [9]:
res, notetext = QwenQnA('what is a startup entrepreneurship?', vectorstore,4,500,qwen05b)

Output()

Question: what is a startup entrepreneurship?

A startup entrepreneur typically involves launching new and disruptive innovation in the 
form of technology-driven solutions or entirely new businesses. These ventures are often 
characterized by rapid growth potential, high risk/high reward balance between success (in
competitive markets) versus failure due to inherent risks involved with running such 
enterprises.

In summary: a start-up entrepreneurship is about taking ownership for creating innovative 
products/services and building something from the ground up in an environment where 
there's significant opportunity but also carries some level of uncertainty.

Completed in: 0:00:54.322737

In [10]:
console.print(notetext)

[
    Document(
        page_content=' and challenging journey filled with both immense rewards and 
potential setbacks.\nI. What is entrepreneurship\nThe world of business is brimming with 
established giants and innovative upstarts. But what drives the creation of these new 
ventures? This is where the concept of entrepreneurship steps in.\nEntrepreneurship is the
process of identifying opportunities, turning ideas into reality, and creating new 
businesses. It’s a captivating blend of innovation, initiative, and calculated 
risk-taking. Entrepreneurs are the trailblazers who:\nSpot a gap in the market: They have 
a keen eye for unmet needs or problems waiting for solutions.\nCraft unique solutions: 
They translate those needs into a product, service, or business model that addresses the 
identified gap.\nEmbrace calculated risks: They invest their time, resources, and effort 
into bringing their vision to life, understanding the potential rewards and challenges 
that lie ahead.\nBuild and manage the business: They assemble a team, secure funding, 
navigate the complexities of running a successful enterprise, and ensure its growth and 
sustainability.\nIn essence, entrepreneurship is about taking ownership and turning ideas 
into action. It’s about being the architect of your own destiny and building something 
from the ground up',
        metadata={
            'source': '/content/What is Entrepreneurship 5 Factors Affecting Ent.txt'
        }
    ),
    Document(
        page_content=' failure in a competitive landscape.\nFocus on Technology: Many 
startups leverage technology to create innovative products or services.\n2. Small business
entrepreneurship\nThe backbone of many communities, small business entrepreneurship 
centers around establishing a local business that caters to a specific niche or geographic
area. Think your friendly neighborhood bakery or a local bookstore. These ventures often 
prioritize:\nStability and Sustainability: Small businesses focus on building a loyal 
customer base and providing reliable goods and services within their community.\nCommunity
Focus: Understanding and catering to the specific needs of the local market is 
paramount.\nLower Risk Profile: While growth is still important, small businesses tend to 
prioritize stability over rapid expansion.\n3. Corporate entrepreneurship 
(Intrapreneurship)\nInnovation doesn’t have to happen outside established organizations. 
Intrapreneurship fosters a culture of innovation within existing companies. Intrapreneurs 
are employees who identify and develop new business ideas or products within the larger 
company structure. They might:\nDevelop new product lines or services.\nIdentify new 
market opportunities.\nStreamline internal processes to improve 
efficiency.\nEntrepreneurial ventures benefit from the resources and infrastructure of the
established company, but navigating internal approval processes and overcoming',
        metadata={
            'source': '/content/What is Entrepreneurship 5 Factors Affecting Ent.txt'
        }
    ),
    Document(
        page_content=', and operational plan. While it’s a dynamic document that can 
evolve as you learn and grow, a well-crafted business plan will help you stay focused and 
attract potential investors.\nAlso read: Crafting an Effective Product Launch Strategy: 
Best Practices and Examples\n3. Secure Funding\nBootstrapping: Many entrepreneurs start by
bootstrapping their businesses using personal savings, loans from family and friends, or 
crowdfunding platforms.\nSeeking Investors: For ventures requiring significant capital, 
exploring options like angel investors or venture capital firms might be necessary. Be 
prepared to present a compelling pitch that highlights the potential of your business.\n4.
Build Your Team\nAssemble Your A-Team: Surround yourself with talented and passionate 
individuals who complement your skillset. Delegate tasks effectively and foster a 
collaborative environment 

In [11]:
res, notetext = QwenQnA('what is the Small business entrepreneurship?', vectorstore,4,500,qwen05b)

Output()

Question: what is the Small business entrepreneurship?

Small Business Entrepreneurship centers on establishing local small enterprises that cater
to specific niche or geographic area, focusing primarily on stability (stability) & 
sustainability (sustainability), community focus (community), lower risk profile (risk). 
It also involves understanding and catering to a particular market. The venture often 
prioritizes the establishment of stable businesses with loyal customers base within their 
locality for long-term growth prospects

Completed in: 0:00:45.735451

In [12]:
console.print(notetext)

[
    Document(
        page_content=' failure in a competitive landscape.\nFocus on Technology: Many 
startups leverage technology to create innovative products or services.\n2. Small business
entrepreneurship\nThe backbone of many communities, small business entrepreneurship 
centers around establishing a local business that caters to a specific niche or geographic
area. Think your friendly neighborhood bakery or a local bookstore. These ventures often 
prioritize:\nStability and Sustainability: Small businesses focus on building a loyal 
customer base and providing reliable goods and services within their community.\nCommunity
Focus: Understanding and catering to the specific needs of the local market is 
paramount.\nLower Risk Profile: While growth is still important, small businesses tend to 
prioritize stability over rapid expansion.\n3. Corporate entrepreneurship 
(Intrapreneurship)\nInnovation doesn’t have to happen outside established organizations. 
Intrapreneurship fosters a culture of innovation within existing companies. Intrapreneurs 
are employees who identify and develop new business ideas or products within the larger 
company structure. They might:\nDevelop new product lines or services.\nIdentify new 
market opportunities.\nStreamline internal processes to improve 
efficiency.\nEntrepreneurial ventures benefit from the resources and infrastructure of the
established company, but navigating internal approval processes and overcoming',
        metadata={
            'source': '/content/What is Entrepreneurship 5 Factors Affecting Ent.txt'
        }
    ),
    Document(
        page_content='title: What is Entrepreneurship - 5 Factors Affecting 
Entrepreneurship\xa0\nauthor: Lynsey PT from Adamo Software | Predict | Jun, 2024 | 
Medium\nurl: 
https://medium.com/predict/what-is-entrepreneurship-5-factors-affecting-entrepreneurship-4
aadad3f161e\n\n\nWhat is Entrepreneurship — 5 Factors Affecting Entrepreneurship\nEver 
dreamt of turning your idea into a reality? Explore the world of entrepreneurship! This 
guide dives into what it means to be an entrepreneur.\nHave you ever dreamt of being your 
own boss, building something from the ground up, and making a positive impact on the 
world? If so, then entrepreneurship might be the path for you! But what exactly is 
entrepreneurship?\nEntrepreneurship is the process of identifying opportunities, creating 
new businesses, and taking calculated risks to bring ideas to life. It’s a dynamic and 
challenging journey filled with both immense rewards and potential setbacks.\nI. What is 
entrepreneurship\nThe world of business is brimming with established giants and innovative
upstarts. But what drives the creation of these new ventures? This is where the concept',
        metadata={
            'source': '/content/What is Entrepreneurship 5 Factors Affecting Ent.txt'
        }
    ),
    Document(
        page_content=', and operational plan. While it’s a dynamic document that can 
evolve as you learn and grow, a well-crafted business plan will help you stay focused and 
attract potential investors.\nAlso read: Crafting an Effective Product Launch Strategy: 
Best Practices and Examples\n3. Secure Funding\nBootstrapping: Many entrepreneurs start by
bootstrapping their businesses using personal savings, loans from family and friends, or 
crowdfunding platforms.\nSeeking Investors: For ventures requiring significant capital, 
exploring options like angel investors or venture capital firms might be necessary. Be 
prepared to present a compelling pitch that highlights the potential of your business.\n4.
Build Your Team\nAssemble Your A-Team: Surround yourself with talented and passionate 
individuals who complement your skillset. Delegate tasks effectively and foster a 
collaborative environment where everyone feels valued and empowered to contribute.\n5. 
Choose a Business Structure\nSole Proprietorship, Partnership, LLC, or Corporation? The 
legal structure you choose for you

In [29]:
res, notetext = QwenQnA('what to do to get started with entrepreneurship?', vectorstore,5,500,qwen05b)

Output()

Question: what to do to get started with entrepreneurship?

To start getting into entrepreneurial ventures:

1. Validate your idea - Market research helps understand if there's an opportunity in this
market.
2. Secure funding – Bootstrapping involves using personal savings, loans from family and 
friends or crowdfunding platforms for startups.

3. Build a team: Assemble talented individuals who complement the skills of entrepreneurs 
to create innovative products/services that can drive economic growth.

4. Choose your business structure - Sole proprietorship is often preferred as it allows 
you control over all aspects while keeping taxes low.
5. Secure funding – Angel investors, venture capitalists or crowdfunding platforms are 
available for startups seeking significant capital investment and support from the 
community of entrepreneurs who share similar goals to yours.

Completed in: 0:00:44.952537

In [ ]:
console.print(notetext)